In [4]:
!pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-7vjg4a3g/unsloth_ca5d1b429bed4795836bc8f554280c63
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-7vjg4a3g/unsloth_ca5d1b429bed4795836bc8f554280c63
  Resolved https://github.com/unslothai/unsloth.git to commit 976d11a10d54383aeb7a692c69e01151a20bfd72
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached https://download.pytorch.org/whl/cu121/xformers-0.0.27.post2-cp310-cp310-manylinux2014_x86_64.whl (20.8 MB)


In [5]:
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [6]:
import torch
import pandas

In [7]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [9]:
from pathlib import Path
import pandas as pd
data_path = Path('drive/MyDrive/LLMResearch/METCLOUD-alldata.csv')
df = pd.read_csv(data_path)
df = df.rename(columns = {'Question':'instruction','Answer':'output'})
df['input'] = ''

In [10]:
df.head(n=2)

,instruction,output,input
0,What is METCLOUD?,METCLOUD is a secure sovereign cloud service p...,
1,How to contact METCLOUD?,"To contact METCLOUD, you can reach out to them...",


In [11]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
    output_column_name = "output",
    conversation_extension = 3, # Select more to handle longer conversations
)

from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

Merging columns:   0%|          | 0/2571 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/2571 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/2571 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/2571 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/2571 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/2571 [00:00<?, ? examples/s]

Standardizing format:   0%|          | 0/2571 [00:00<?, ? examples/s]

In [12]:
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    default_system_message = "You are a Cyber helpful assistant. Below are some examples of questions you are able to answer, and their correct answers"
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/2571 [00:00<?, ? examples/s]

In [13]:
dataset[0]

{'conversations': [{'content': 'What is METCLOUD?', 'role': 'user'},
  {'content': "METCLOUD is a secure sovereign cloud service provider that specializes in offering digital modernization through advanced cybersecurity and artificial intelligence. It is designed to support businesses in adopting next-generation technologies for cloud computing and cybersecurity, ensuring that they stay secure, effective, and efficient. METCLOUD's approach is tailored to meet the unique needs of businesses, with a focus on a people-first strategy. The platform is scalable, making it suitable for small to medium-sized enterprises, and it has been recognized for its excellence in the field, including being named the Cybersecurity Firm of the Year by Finance Monthly in the 2021 FinTech Awards.",
   'role': 'assistant'},
  {'content': 'Can you explain what log file analysis entails in the context of digital forensics?',
   'role': 'user'},
  {'content': "Certainly! Log file analysis in digital forensics in

In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #max_steps = 60,
        num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/2571 [00:00<?, ? examples/s]

In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,571 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 321
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.244600
2,1.162500
3,1.219500
4,1.031600
5,0.963100
6,0.756200
7,0.728100
8,0.747100
9,0.679100
10,0.716100


In [16]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "What is METCLOUD? Who is their CEO?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


METCLOUD is a leading cybersecurity company that specializes in providing cloud security solutions. The CEO of METCLOUD is Mr. Sohail Shah, who is a seasoned executive with extensive experience in the cybersecurity and technology industries. Under his leadership, METCLOUD has grown to become a prominent player in the field of cloud security, offering innovative solutions to protect organizations' data and infrastructure in the cloud environment.<|eot_id|></s>


Saving and Quantizing the model

In [17]:
#model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
#model_name = "Meta-Llama-3.1-8B-Instruct-bnb-4bit"
save_dir = f'drive/MyDrive/LLMResearch/Finetuned/{model_name.split("/")[-1]}'
model.save_pretrained_gguf(save_dir,tokenizer,quantization_method="q4_k_m")

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 4.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 60.18 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:01<00:00, 26.67it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting mistral model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at drive/MyDrive/LLMResearch/Finetuned/mistral-7b-instruct-v0.3-bnb-4bit into bf16 GGUF format.
The output location will be ./drive/MyDrive/LLMResearch/Finetuned/mistral-7b-instruct-v0.3-bnb-4bit/unsloth.BF16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: mistral-7b-instruct-v0.3-bnb-4bit
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00003.safeten

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: ./drive/MyDrive/LLMResearch/Finetuned/mistral-7b-instruct-v0.3-bnb-4bit/unsloth.Q4_K_M.gguf
Unsloth: Saved Ollama Modelfile to drive/MyDrive/LLMResearch/Finetuned/mistral-7b-instruct-v0.3-bnb-4bit/Modelfile
